# Testing downloads from APPLAUSE

We ned to download two kinds of data: plate scans, and source tables. Analysis requires two plates and four source tables:

- plate scan 1 
- plate scan 2
- sources 1
- sources calib 1
- sources 2
- sources calib 2

In [1]:
import os
import shutil
from importlib import reload
from urllib.parse import urlsplit
import requests
import json

import pyvo as vo

from astropy.io import fits
from astropy.utils.data import download_file
from astropy.table import Table

from settings import DATAPATH

## Functions

In [2]:
def download_scan(table, plate_id, test=False):

    # get URL for this plate ID 
    mask = table['plate_id'] == plate_id
    table_1 = table[mask]
    url_1 = table_1['filename_scan'][0]
    
    # get file name from URL
    parsed_url = urlsplit(url_1)
    filename = parsed_url.path.split('/')[-1]

    # Download the file and get the local filename/path
    local_file_path = download_file(url_1)
    print(f"File downloaded to: {local_file_path}")

    # Move and rename file into data directory
    new_path = shutil.move(local_file_path, os.path.join(DATAPATH, filename))
    print("new path is: ", new_path)

    # TEST: open the local FITS file using the obtained path
    if test:
        with fits.open(new_path) as hdul:
            # Access the header information or data
            header_info = hdul[0].header
            print(f"TEST: telescope used: {header_info.get('TELESCOP')}")
            print(f"TEST: date: {header_info.get('DATE-AVG')}")
            
#     images_dict = pickle.load(open("image_library.py", "rb" ))
#     images_dict[str(plate_id)] = filename
#     pickle.dump(images_dict, open( "image_library.py", "wb" ) )
    
    _file_path = os.path.join(os.getcwd(), 'image_library.py')
    f = open(_file_path, 'r')
    text = json.load(f)
    
    print(text)

In [ ]:
def download_sources_table(plate_id, calib=False):
    # setup
    name = 'APPLAUSE',
    url = 'https://www.plate-archive.org/tap'
    token = 'Token 2e2225623fdf3e89aef67a04415ade1d9369193e'

    # assemble query string
    qstr = 'SELECT * FROM applause_dr4.source'
    if calib:
        qstr = qstr + '_calib'
    qstr = qstr + ' WHERE plate_id = ' + str(plate_id) + ' ORDER BY source_id'

#     print('\npyvo version %s \n' % vo.__version__)
#     print('TAP service %s \n' % name)

    tap_session = requests.Session()
    tap_session.headers['Authorization'] = token

    tap_service = vo.dal.TAPService(url, session=tap_session)

    lang = 'PostgreSQL'

    job = tap_service.submit_job(qstr, language=lang)
    job.run()

    job.wait(phases=["COMPLETED", "ERROR", "ABORTED"], timeout=30.)

    job.raise_if_error()
    results = job.fetch_result()
    
    return results.to_table()

## Downloading plate scans

In [3]:
catalog_applause = Table.read('footprints_5.csv', format='ascii.csv')

plate_id = 9319

In [4]:
download_scan(catalog_applause, plate_id, test=True)

File downloaded to: /var/folders/y5/zcwj7zc17g19nggvb1vx4rc80000gn/T/astropy-download-65848-ad_io3xl
new path is:  /Users/busko/Projects/VASCO_data/footprints/GS00768_x.fits
TEST: telescope used: Grosser Schmidt-Spiegel
TEST: date: 1956-12-03T20:27:18


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

## Downloading source tables

In [ ]:
results = download_sources_table(9320)

results